In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Symbols

---


| Variable                                 	| Unit of Measurement 	|
|------------------------------------------	|----------------------	|
| $k$, permeability of the liver           	| $m^2$                	|
| $\mu_s$, viscosity of embolizing solution 	| $Pa \cdot s$         	|
| $\mu_o$, viscosity of blood               	| $Pa \cdot s$         	|
| distances                                	| $m$                  	|

| **Symbol**                  | **Description**                                | **Unit**           |
|-----------------------------|------------------------------------------------|--------------------|
| $ k $                     | Permeability of the liver                      | $ \mathrm{m}^2 $ |
| $ \mu_s $                 | Viscosity of embolizing solution               | $ \mathrm{Pa} \cdot \mathrm{s} $ |
| $ \mu_o $                 | Viscosity of blood                             | $ \mathrm{Pa} \cdot \mathrm{s} $ |
| $ \Omega $                | 3D Bulk Domain                                 | Dimensionless      |
| $ \Sigma $                | Cylindrical Vessel                             | Dimensionless      |
| $ \Gamma $                | Lateral Boundary of $ \Sigma $               | Dimensionless      |
| $ \Gamma_0, \Gamma_S $    | Top and Bottom Faces of $ \Sigma $           | Dimensionless      |
| $ \lambda(s) $            | $\mathcal{C}^2$-regular Curve (Centerline)    | $ \mathrm{m} $ (Parameter $ s $) |
| $ s $                     | Arc-length Parameter along $ \lambda $        | $ \mathrm{m} $   |
| $ \Lambda $               | Centerline of $ \Sigma $                      | Dimensionless      |
| $ \mathcal{D}(s) $        | Cross-section of $ \Sigma $                   | $ \mathrm{m}^2 $ |
| $ \partial \mathcal{D}(s) $| Boundary of Cross-section $ \mathcal{D}(s) $  | $ \mathrm{m} $   |
| $ u_{\oplus} $            | Fluid Potential in $ \Omega $ (Exterior)       | $ \mathrm{Pa} $  |
| $ u_{\ominus} $           | Fluid Potential in $ \Lambda $ (Interior)      | $ \mathrm{Pa} $  |
| $ \beta $                 | Coupling Coefficient                            | $ \frac{\mathrm{m}^3}{\mathrm{s} \cdot \mathrm{Pa}} $ |
| $ \kappa $                | Permeability Coefficient                        | $ \frac{\mathrm{m}^2}{\mathrm{s} \cdot \mathrm{Pa}} $ |
| $ g $                     | Source Term                                    | $ \frac{\mathrm{m}}{\mathrm{s}} $ |
| $ \alpha $                | Diffusion Coefficient                           | $ \frac{\mathrm{m}^2}{\mathrm{s} \cdot \mathrm{Pa}} $ |
| $ \Delta $                | Laplacian Operator                              | $ \frac{1}{\mathrm{m}^2} $ |
| $ \delta_{\Lambda} $      | Dirac Measure on $ \Lambda $                  | $ \frac{1}{\mathrm{m}^2} $ |
| $ f $                     | Forcing Term                                    | $ \frac{1}{\mathrm{s}} $ |
| $ \bar{u}_{\oplus} $      | Averaged Fluid Potential in $ \Omega $         | $ \mathrm{Pa} $  |
| $ \bar{u}_{\ominus} $     | Averaged Fluid Potential in $ \Lambda $        | $ \mathrm{Pa} $  |

In [2]:
# @title Install nonstandard libraries
%%capture
!pip install ipywidgets
!pip install vtk
!pip install meshio
!pip install pyvista
!pip install Rtree

import os, re

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

In [3]:
WD_PATH = "/content/drive/MyDrive/Research/3D-1D"

import sys, os
sys.path.append(os.path.join(WD_PATH, 'modules'))

import FEMEdge

import meshio
import scipy
import copy
import vtk
import json
import numpy as np
import matplotlib.pyplot as plt
import datetime
import networkx as nx
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import cKDTree
import importlib
from dolfin import *
from vtk.util.numpy_support import vtk_to_numpy
from xii import *
from graphnics import *

In [ ]:
# @title Define G = serena data graph
G = FenicsGraph()
ind = 0
branch_points = {}

for n in range(29):
    file_path = os.path.join(WD_PATH, 'data', 'pv_json1', f'Centerline_{str(n)}.mrk.json')
    f = open(file_path)
    data = json.load(f)
    f.close()

    # get coords + radius at each point
    points = data['markups'][0]['controlPoints']
    radius = data['markups'][0]['measurements'][3]['controlPointValues']
    G.add_nodes_from(range(ind - n, ind + len(points) - n))

    # check if first coord is branch point from previous centerlines
    v1 = 0
    for key, val in branch_points.items():
        if points[0]['position'] == val:
            v1 = key
            break

    # add coords and radius to nodes fenics graph
    v2 = ind - n + 1
    pos_v1 = points[0]['position']
    pos_v2 = points[1]['position']
    G.nodes[v1]["pos"] = pos_v1
    G.nodes[v2]["pos"] = pos_v2
    G.nodes[v1]["radius"] = radius[0]
    G.nodes[v2]["radius"] = radius[1]
    # add edge to fenics graph
    G.add_edge(v1, v2)

    for i in range(len(points)-2):
        v1 = ind - n + 1 + i
        v2 = v1 + 1
        # convert coordinates from mm to meters (divide by 1000)
        pos_v1 = [coord / 1000 for coord in points[0]['position']]
        pos_v2 = [coord / 1000 for coord in points[1]['position']]
        pos_v1 = points[i + 1]['position']
        pos_v2 = points[i + 2]['position']
        G.nodes[v1]["pos"] = pos_v1
        G.nodes[v2]["pos"] = pos_v2
        G.nodes[v1]["radius"] = radius[i + 1]
        G.nodes[v2]["radius"] = radius[i + 2]
        G.add_edge(v1, v2, radius=radius[i+1])

    # store last point as a branch point
    ind += len(points)
    branch_points.update({ind-n-1: pos_v2})

In [ ]:
# @title Define G = .vtk domain read
import vtk

def read_vtk(file_path):
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    output = reader.GetOutput()

    G = FenicsGraph()

    damage_array = output.GetPointData().GetArray("Damage")
    for i in range(output.GetNumberOfPoints()):
        point = output.GetPoint(i)
        damage_value = damage_array.GetValue(i)
        G.add_node(i, pos=tuple(point), damage=damage_value)

    radius_array = output.GetCellData().GetArray("Radius")
    for i in range(output.GetNumberOfCells()):
        cell = output.GetCell(i)
        point_ids = [cell.GetPointId(j) for j in range(cell.GetNumberOfPoints())]
        for j in range(len(point_ids) - 1):
            u = point_ids[j]
            v = point_ids[j + 1]
            radius_value = radius_array.GetValue(i) if radius_array else None
            G.add_edge(u, v, radius=radius_value)

    return G

# Usage
file_path = WD_PATH + '/data/vtk/sortedDomain.vtk'
# file_path = WD_PATH + '/oncopigReferenceData/ZPAF23S018/20230531/vesselNetwork_upDated.vtk'
G = read_vtk(file_path)

In [ ]:
def filter_small_components(G, threshold=20):
  """
  Removes connected components from the graph G that have a total length
  (sum of edge lengths) less than the specified threshold.

  Args:
    G: The networkx graph to process.
    threshold: The minimum total length for a component to be kept.
  """

  # Convert to undirected graph
  G_undirected = G.to_undirected()

  # Find connected components
  components = list(nx.connected_components(G_undirected))

  # Calculate total length for each component
  for component in components:
    total_length = 0
    for u, v in G_undirected.subgraph(component).edges():
      pos_u = G_undirected.nodes[u]['pos']
      pos_v = G_undirected.nodes[v]['pos']
      length = ((pos_u[0] - pos_v[0])**2 +
                (pos_u[1] - pos_v[1])**2 +
                (pos_u[2] - pos_v[2])**2) ** 0.5
      total_length += length

    # Remove component if total length is below threshold
    if total_length < threshold:
      G.remove_nodes_from(component)  # Remove from the original directed graph

# Apply the filtering
filter_small_components(G)

# @title Clean graph (only for raw .vtk domain reads)
def cleanup_graph(G):
    G.remove_nodes_from(list(nx.isolates(G)))
    G = nx.convert_node_labels_to_integers(G)

    # Merge close nodes
    positions = nx.get_node_attributes(G, 'pos')
    positions_array = np.array(list(positions.values()))
    tree = cKDTree(positions_array)
    merged_nodes = set()

    for node, pos in positions.items():
        # Find nearby nodes within the threshold distance
        nearby_nodes = tree.query_ball_point(pos, 1.0e-4)

        # Merge with the first nearby node (if any) that hasn't been merged yet
        for other_node in nearby_nodes:
            if other_node != node and other_node not in merged_nodes:
                G = nx.contracted_nodes(G, node, other_node)
                merged_nodes.add(other_node)
                break  # Stop after merging with one node

    # Set zero radii to 0.1
    for node in G.nodes():
        if G.nodes[node]['radius'] == 0:
            G.nodes[node]['radius'] = 0.1

    return G

G = cleanup_graph(G)

In [ ]:
# @title Define test graph
# Create the FenicsGraph
G = FenicsGraph()

# Add nodes and their attributes
node_data = {
    0: {'pos': [8, 20, 15], 'radius': 1},
    1: {'pos': [10, 20, 15], 'radius': 1},
    2: {'pos': [11, 20, 15], 'radius': 1},
    3: {'pos': [13, 20, 15], 'radius': 1},
    4: {'pos': [15, 20, 15], 'radius': 1},
    5: {'pos': [16, 20, 15], 'radius': 1},
    6: {'pos': [18, 20, 15], 'radius': 1},
    7: {'pos': [20, 20, 15], 'radius': 1},
    8: {'pos': [21, 20, 15], 'radius': 1},
    9: {'pos': [23, 20, 15], 'radius': 1},
    10: {'pos': [33, 37, 15], 'radius': 1},
    11: {'pos': [32, 35, 15], 'radius': 1},
    12: {'pos': [31, 33, 15], 'radius': 1},
    13: {'pos': [30, 31, 15], 'radius': 1},
    14: {'pos': [29, 29, 15], 'radius': 1},
    15: {'pos': [27, 28, 15], 'radius': 1},
    16: {'pos': [26, 26, 15], 'radius': 1},
    17: {'pos': [25, 24, 15], 'radius': 1},
    18: {'pos': [24, 22, 15], 'radius': 1},
    19: {'pos': [23, 20, 15], 'radius': 1},
    20: {'pos': [33, 3, 15], 'radius': 1},
    21: {'pos': [32, 5, 15], 'radius': 1},
    22: {'pos': [31, 7, 15], 'radius': 1},
    23: {'pos': [30, 9, 15], 'radius': 1},
    24: {'pos': [29, 11, 15], 'radius': 1},
    25: {'pos': [27, 12, 15], 'radius': 1},
    26: {'pos': [26, 14, 15], 'radius': 1},
    27: {'pos': [25, 16, 15], 'radius': 1},
    28: {'pos': [24, 18, 15], 'radius': 1},
    29: {'pos': [23, 20, 15], 'radius': 1}
}

for i in range(9):  # Connect nodes between V1 and C
    G.add_edge(i, i + 1)

for i in range(10, 19):  # Connect nodes between V2 and C
    G.add_edge(i+1, i)

for i in range(20, 29):  # Connect nodes between V3 and C
    G.add_edge(i+1, i)

# Set node attributes
for node_id, attributes in node_data.items():
    for attr_name, attr_value in attributes.items():
        G.nodes[node_id][attr_name] = attr_value

for i in range(9):  # Connect nodes between V1 and C
    G.add_edge(i, i + 1)

# pos_t = {idx: data['pos'][:2] for idx, data in node_data.items()}  # 2D position (x, y)
# nx.draw(G, pos_t, with_labels=True, node_color='lightblue', node_size=500, font_size=10, font_color='black')

In [4]:
#@title Define test graph 2
# Import the necessary library
G = FenicsGraph()

# Define the node labels and their corresponding integer IDs
node_mapping = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E',
    5: 'F',
    6: 'G',
    7: 'H'
}

# Define the coordinates for each node (x, y, z)
node_coords = {
    0: [0, 20, 15],    # A
    1: [10, 20, 15],   # B
    2: [22, 13, 15],   # C
    3: [22, 28, 15],   # D
    4: [15, 5, 15],    # E
    5: [15, 35, 15],   # F
    6: [38, 5, 15],    # G
    7: [38, 35, 15]    # H
}

# Define the edges along with their radii
edges_with_radii = [
    (0, 1, 4),  # AB
    (1, 2, 3),  # BC
    (1, 3, 3),  # BD
    (2, 4, 2),  # CE
    (2, 6, 3),  # CG
    (3, 5, 2),  # DF
    (3, 7, 3)   # DH
]

# Create the FenicsGraph object
G = FenicsGraph()

# Add nodes to the graph with their positions
for node_id, coord in node_coords.items():
    G.add_node(node_id, pos=coord)

# Add edges to the graph with their radii
for u, v, radius in edges_with_radii:
    G.add_edge(u, v, radius=radius)

In [8]:
importlib.reload(FEMEdge)

<module 'FEMEdge' from '/content/drive/MyDrive/Research/3D-1D/modules/FEMEdge.py'>

In [9]:
k = 9.44e-3
mu_o = 4.5e-3
P_cvp = 5 #mmHg
P_in = 100 #mmHg

test = FEMEdge.FEMEdge(
  G,
  kappa = 1.0,
  alpha = k / mu_o,
  beta = 6.0e+4 * (k / mu_o),
  gamma = 0.2,
  del_Omega = P_cvp * 133.322, # Pa
  P_infty = P_in * 133.322, # Pa
  theta = 1.0,
  P_sink = P_in * 133.322,
  Omega_box = [40, 40, 30, 0, 0, 0]
)

Averaging over 14 cells: 100%|██████████| 14/14 [00:00<00:00, 472.14it/s]


In [10]:
test.save_vtk(os.path.join(WD_PATH, 'perfusion_results', 'test_rh_ex8'))